In [1]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import re

In [2]:
url = 'https://www.pro-football-reference.com/boxscores/202109120nor.htm'
response = requests.get(url)
response

<Response [200]>

- 1 SOUP
- 2 div_grid [0]
- 3 div [0]
- 4 div_wrap [0]
- 5 soup_wrap4 [4]
- 6 soup4 : reparse soup_wrap4
- 7 wx

In [ ]:
#1
soup = BeautifulSoup(response.content, 'html.parser')
#2
div_grid = soup.find_all('div', class_='content_grid')[0]
#3
div = div_grid.find_all('div')[0]
#4
div_wrap = div.find_all('div')[0]
#5
soup_wrap4 = div_wrap.contents[4]
#6
soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
#7
wx = soup4.find_all('td','center')[6]

In [ ]:
wx.contents

In [ ]:
wx.contents[0]

In [ ]:
content = wx.contents[0]

In [ ]:
content = str(content)

In [ ]:
type(content)

## So now we have options...

In [ ]:
# regex to return numerical string at start of object
temp = r'^\d{1,2}'
re.findall(temp,content)

In [ ]:
wind = r'\d{1,2}[\smph]'
# etc....

## Or, a different approach

In [ ]:
wx_digits = r'\d{1,2}'
re.findall(wx_digits,content)

In [ ]:
re.findall(wx_digits,content)[0]

In [ ]:
re.findall(wx_digits,content)[0]

In [ ]:
re.findall(wx_digits,content)[1]

In [ ]:
re.findall(wx_digits,content)[2]

## Ok, now we increase the scale to WEEKLY

In [3]:
url = 'https://www.pro-football-reference.com/years/2022/week_1.htm'

In [4]:
response = requests.get(url)
response

<Response [200]>

In [33]:
response

<Response [429]>

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
html_links = soup.find_all('td', class_='right gamelink')

In [7]:
html_links[0].find('a')['href']

'/boxscores/202209080ram.htm'

## List of Games

In [8]:
[item.find('a')['href'] for item in html_links]

['/boxscores/202209080ram.htm',
 '/boxscores/202209110atl.htm',
 '/boxscores/202209110car.htm',
 '/boxscores/202209110chi.htm',
 '/boxscores/202209110cin.htm',
 '/boxscores/202209110det.htm',
 '/boxscores/202209110htx.htm',
 '/boxscores/202209110mia.htm',
 '/boxscores/202209110nyj.htm',
 '/boxscores/202209110was.htm',
 '/boxscores/202209110crd.htm',
 '/boxscores/202209110min.htm',
 '/boxscores/202209110oti.htm',
 '/boxscores/202209110sdg.htm',
 '/boxscores/202209110dal.htm',
 '/boxscores/202209120sea.htm']

In [9]:
games = [item.find('a')['href'] for item in html_links]

In [10]:
games

['/boxscores/202209080ram.htm',
 '/boxscores/202209110atl.htm',
 '/boxscores/202209110car.htm',
 '/boxscores/202209110chi.htm',
 '/boxscores/202209110cin.htm',
 '/boxscores/202209110det.htm',
 '/boxscores/202209110htx.htm',
 '/boxscores/202209110mia.htm',
 '/boxscores/202209110nyj.htm',
 '/boxscores/202209110was.htm',
 '/boxscores/202209110crd.htm',
 '/boxscores/202209110min.htm',
 '/boxscores/202209110oti.htm',
 '/boxscores/202209110sdg.htm',
 '/boxscores/202209110dal.htm',
 '/boxscores/202209120sea.htm']

## List of homes

In [11]:
home_re = r'[a-z]{3}'

In [12]:
home = re.findall(home_re,games[0])[3]

In [13]:
type(home)

str

In [14]:
home

'ram'

In [15]:
homes = [re.findall(home_re,i)[3] for i in games]

In [ ]:
# homes_2 = []
# for i in games:
#     home_re = r'[a-z]{3}'
#     home = re.findall(home_re,i)[3]
#     homes.append(home)

In [16]:
homes

['ram',
 'atl',
 'car',
 'chi',
 'cin',
 'det',
 'htx',
 'mia',
 'nyj',
 'was',
 'crd',
 'min',
 'oti',
 'sdg',
 'dal',
 'sea']

## List of seasons

In [17]:
date_re = r'[0-9]{4}'

In [18]:
re.findall(date_re,games[0])[0]

'2022'

In [19]:
season = re.findall(date_re,games[0])[0]

In [20]:
season

'2022'

In [21]:
seasons = [re.findall(date_re,i)[0] for i in games]

In [22]:
seasons

['2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022']

## List of weeks

In [23]:
weeks = [1 for i in games]

In [24]:
weeks

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

## List of temps

In [25]:
temps = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    temp = re.findall(wx_digits,content)[0]
    temps.append(temp)

In [26]:
temps

['20',
 '20',
 '80',
 '69',
 '76',
 '20',
 '20',
 '92',
 '73',
 '74',
 '20',
 '20',
 '75',
 '20',
 '20',
 '74']

In [ ]:
df = pd.DataFrame({
    'season':seasons,
    'week':weeks,
    'home':homes,
    'temp':temps
})

In [ ]:
df

## List of winds

In [30]:
winds = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    wind = re.findall(wx_digits,content)[2]
    winds.append(wind)

IndexError: list index out of range

In [ ]:
winds

## List of hums

In [ ]:
hums = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    hum = re.findall(wx_digits,content)[1]
    hums.append(hum)

In [ ]:
hums

In [ ]:
df = pd.DataFrame({
    'schedule_season':seasons,
    'schedule_week':weeks,
    'team_home':homes,
    'weather_temperature':temps,
    'weather_wind_mph':winds,
    "weather_humidity":hums
})

In [ ]:
df